In [ ]:
!gdown '1pRE91UrrutDNPdSCzjsf64DgiKl25k8d'
!unzip "/content/CSVs.zip" -d "/content/CSVs"
!rm -rf '/content/CSVs.zip'

In [ ]:
import pandas as pd
import numpy as np
import glob

In [ ]:
files = glob.glob("./CSVs/*.csv")

row = 0
mult = int(5e3)
seq_rest = 0
seq_active = 0

# Nome de cada coluna do dataframe (para referência)
nomes = ['Fp1','Fp2','F3','F4','F7','F8','T3','T4','C3','C4','T5','T6','P3','P4','O1','O2','Fz','Cz','Pz','A2-A1','ECG','State']

# A quantidade de cada tipo de segmento definida com base no artigo
rest_seqs = np.zeros((601,10000,len(nomes)))
acti_seqs = np.ones((180,10000,len(nomes)))

for file in files:
  df = pd.read_csv(file)
  subject = int(file[-5])-1
  df['State'] = subject

  aux = df.to_numpy()
  
  for i in range(1000000):
    # Quando a segmentação resultar em um segmento com menos de 10k de amostras ou resulte em
    # um vetor vazio (o indice de início é maior que o vetor), acaba a segmentação deste arquivo
    # e vai para o próximo.
    if ~np.any(aux[i*mult:(i+2)*mult,:]) or aux[i*mult:(i+2)*mult,:].shape[0] != 10000:
      break
    if subject==1:
      acti_seqs[seq_active,:,:] = aux[i*mult:(i+2)*mult,:]
      seq_active = seq_active+1
    else:
      rest_seqs[seq_rest,:,:] = aux[i*mult:(i+2)*mult,:]
      seq_rest = seq_rest+1


In [ ]:
def segmenta(sinal, passo, dimensoes):
  #Inputs
  #- sinal: sinal a ser segmentado
  #- passo: quantas amostras irá avançar de um segmento para outro
  #- dimensoes: Dimensões esperada do output

  #Output
  #- segmentado: Sinal segmentado em segmentos de 2*passo de duração e com
  # overlap de 1 passo entre eles

  segmentado = np.zeros(dimensoes)
  cont = 0
  for linha in range(sinal.shape[0]):
      sinal_s = sinal[linha,:,:]
      for i in range(1000000):
          if ~np.any(sinal_s[i*passo:(i+2)*passo,:]) or sinal_s[i*passo:(i+2)*passo,:].shape[0] != int(2*passo):
              break
          segmentado[cont,:,:] = sinal_s[i*passo:(i+2)*passo,:]
          cont = cont+1
  lim = np.where(segmentado[:,0,0]==0)[0][0]
  segmentado = segmentado[:lim,:,:]
  return segmentado

In [ ]:
# geral = np.concatenate((acti_seqs,rest_seqs),axis=0)
# np.save("Unico",geral)

In [ ]:
# Realiza o processo 5x para obter 5 datasets diferentes para cada treinamento
for i in range(1):
    rest_train = list(np.random.choice(601,480,replace=False))
    rest_test = [x for x in range(601) if x not in rest_train]

    # Por conta da diferença entre as quantidades de cada estado, a amostragem
    # dos segmentos ativos deveria ser com reposição. No momento, ela está sendo
    # feita de modo que cada um dos 180 segmentos tenha chance de ser pego até 2x
    active_train = list(np.random.choice(180,144,replace=False))
    active_test = [x for x in range(180) if x not in active_train]

    # # Duplica os segmentos ativos
    # aux = np.copy(acti_seqs)
    # aux = np.concatenate((acti_seqs,aux))

    rest_train_f = rest_seqs[rest_train,:,:]
    active_train_f = acti_seqs[active_train,:,:]
    rest_test_f = rest_seqs[rest_test,:,:]
    active_test_f = acti_seqs[active_test,:,:]

    train = np.concatenate((rest_train_f,active_train_f),axis=0)
    test = np.concatenate((rest_test_f,active_test_f),axis=0)

    name_train = 'Train_Fold'+str(i)
    name_test = 'Test_Fold'+str(i)
    # np.save(name_train,train)
    # np.save(name_test,test)
    

In [ ]:
print(f'Nº de segmentos\nAtivo: {train.shape[0]}\nRepouso: {test.shape[0]}')